In [1]:
import numpy as np
from lib.pde_find2 import PDEFind
from IPython.display import display, Math

In [2]:
raw_data = np.load("data/3.npz")

u = raw_data["u"]
v = raw_data["v"]
x = raw_data["x"]
y = raw_data["y"]
t = raw_data["t"]

In [3]:
pdefind = PDEFind(
    var_labels=(["x", "y", "t"], ["u", "v"]),
    polynomial_degree=3,
    order=2,
    periodic=True,
)

In [4]:
u, v, x, y, t = pdefind.subsample_data(u, v, x, y, t, factors=[3, 3, 2])


In [5]:
pdefind.add_grid(x, y, t)

In [11]:
term_regex = r"^(u|v)*(_{(x|y)+})?(v)*$"

library, labels = pdefind.create_library(u, v, term_regex=term_regex)

display(Math(r"\\".join(labels)))

<IPython.core.display.Math object>

In [7]:
u_t = np.gradient(u, pdefind.time_grid, axis=-1)
v_t = np.gradient(v, pdefind.time_grid, axis=-1)

In [8]:
algorithm = "tlsq"
cutoff = 1e-4
iterations = 500
max_terms = 10

In [9]:
coef_u, alpha_u = pdefind.solve_regression(
    library,
    u_t,
    algorithm=algorithm,
    cutoff=cutoff,
    iterations=iterations,
    num_term_limit=max_terms,
)

Solving using tlsq regression
Library size: (69, 86, 86, 101), Target size: 746996


TLSQ Iterations:  28%|██▊       | 141/500 [00:53<02:16,  2.63it/s, #nzz_terms=8, cutoff=0.0001] 

# of non-zero terms: 8


<IPython.core.display.Math object>

In [13]:
display(Math(pdefind.latex_string(coef_u, labels, "u")))

<IPython.core.display.Math object>

In [10]:
coef_v, alpha_v = pdefind.solve_regression(
    library,
    v_t,
    algorithm=algorithm,
    cutoff=cutoff,
    iterations=iterations,
    num_term_limit=max_terms,
)

Solving using tlsq regression
Library size: (69, 86, 86, 101), Target size: 746996


TLSQ Iterations:  26%|██▌       | 128/500 [00:41<02:01,  3.05it/s, #nzz_terms=9, cutoff=0.0001] 

# of non-zero terms: 9


<IPython.core.display.Math object>

In [14]:
display(Math(pdefind.latex_string(coef_v, labels, "v")))

<IPython.core.display.Math object>

In [12]:
print(pdefind.python_string(coef_u, labels, "u"))
print(pdefind.python_string(coef_v, labels, "v"))

du/dt = +0.762*u+0.108*u_xx+0.105*u_yy-0.735*u*u*u+0.671*u*u*v-0.738*u*v*v+0.287*v+0.67*v*v*v
dv/dt = -0.28*u-0.679*u*u*u-0.727*u*u*v-0.679*u*v*v+0.757*v+0.118*v_xx+0.106*v_yy-0.74*v*v*v-0.0353*v*v*v_xx
